In [24]:
%run plot.py

[simple full uncrelated] Chi2/ndf for D0 vs model tamu_d0_rebin: 0.5698

SYSTEMATIC ERROR CORRELATION TEST RESULTS SUMMARY
0_Baseline                | chi2/ndf = 0.5700
1_All_Uncorrelated        | chi2/ndf = 0.5698
2_Supplement1_All_Interbin_Corr | chi2/ndf = 0.5805
3_Supplement2_Intrabin_Corr | chi2/ndf = 0.5565
4_Supplement3_Full_Corr   | chi2/ndf = 0.4391
5_Partial_Intrabin_Corr   | chi2/ndf = 0.5631
6_Reso_Only               | chi2/ndf = 0.8403
[simple full uncrelated] Chi2/ndf for D0 vs model catania_d0_rebin: 13.8072

SYSTEMATIC ERROR CORRELATION TEST RESULTS SUMMARY
0_Baseline                | chi2/ndf = 13.4596
1_All_Uncorrelated        | chi2/ndf = 13.8072
2_Supplement1_All_Interbin_Corr | chi2/ndf = 18.3620
3_Supplement2_Intrabin_Corr | chi2/ndf = 12.5592
4_Supplement3_Full_Corr   | chi2/ndf = 11.9418
5_Partial_Intrabin_Corr   | chi2/ndf = 12.9926
6_Reso_Only               | chi2/ndf = 18.7270
[simple full uncrelated] Chi2/ndf for D0 vs model langevin_d0_rebin: 298.1132

SYST

Warning in <TROOT::Append>: Replacing existing TH1: d0 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: lc (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: tamu_lc (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: tamu_d0 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: catania_d0 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: catania_lc (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: langevin_lc (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: langevin_d0 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: htl_lc (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: htl_d0 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: latQCD_lc (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: latQCD_d0

In [54]:
%run '../Nsigma/nsigma.py'
# !root -x -q 'TestNsigma.C'
# !root -x -q 'significance_DoubleRatioLcD0.C'


===== Test combination: Particle=lc, Method=binBybin-difference, Case=reso CORR, fit & frac UNC, Combine non-strange=True =====
Method: binBybin-difference, Particle: lc, Correlation case: reso CORR, fit & frac UNC, Combine non-strange: True
Comparison: Lambda_c vs (D0+D+)
Lambda_c bins: [4, 5, 6, 8, 12, 24]
Reference particle bins: [4, 5, 6, 7, 8, 10, 12, 16, 24]
Original bins match target bins, no rebinning needed
Rebinning completed: original bins [4, 5, 6, 7, 8, 10, 12, 16, 24] -> target bins [4, 5, 6, 8, 12, 24]
Test result: Success

===== Test combination: Particle=lc, Method=binBybin-difference, Case=reso CORR, fit & frac UNC, Combine non-strange=False =====
Method: binBybin-difference, Particle: lc, Correlation case: reso CORR, fit & frac UNC, Combine non-strange: False
Comparison: Lambda_c vs D0
Lambda_c bins: [4, 5, 6, 8, 12, 24]
Reference particle bins: [4, 5, 6, 7, 8, 10, 12, 16, 24]
Original bins match target bins, no rebinning needed
Rebinning completed: original bins [4

In [3]:
import ROOT
import sys

def adjust_reso_syst_and_recalc_tot(input_file_path, output_file_path):
    """
    处理ROOT文件：
    1. 将reso_syst所有数据点的y误差设置为中心值（y值）的0.2%
    2. 重新计算tot_syst（总系统误差）：tot_syst = sqrt(fit_syst² + reso_syst² + fd_syst²)
    3. 保存修改后的数据到新文件
    
    参数:
        input_file_path: 输入ROOT文件路径（包含reso_syst/fit_syst/fd_syst/tot_syst等对象）
        output_file_path: 输出ROOT文件路径（保存修改后的数据）
    """
    # 初始化ROOT批处理模式（无GUI）
    ROOT.gROOT.SetBatch(True)
    
    # 1. 打开输入文件
    try:
        in_file = ROOT.TFile.Open(input_file_path, "READ")
        if not in_file or in_file.IsZombie():
            raise RuntimeError(f"无法打开输入文件: {input_file_path}")
        print(f"成功打开输入文件: {input_file_path}")
    except Exception as e:
        print(f"打开文件失败: {e}")
        sys.exit(1)
    
    # 2. 检查必要对象是否存在
    required_objs = ["reso_syst", "fit_syst", "fd_syst", "tot_syst"]
    obj_dict = {}
    for obj_name in required_objs:
        obj = in_file.Get(obj_name)
        if not obj or not isinstance(obj, ROOT.TGraphAsymmErrors):
            print(f"错误：输入文件中未找到有效对象 {obj_name} (TGraphAsymmErrors类型)")
            in_file.Close()
            sys.exit(1)
        obj_dict[obj_name] = obj
        print(f"加载对象 {obj_name}，数据点数量: {obj.GetN()}")
    
    # 3. 处理reso_syst：设置y误差为中心值的0.2%
    reso_graph = obj_dict["reso_syst"]
    n_points = reso_graph.GetN()
    print(f"\n开始修改reso_syst的y误差（设置为y值的0.2%）...")
    
    # 创建修改后的reso_syst副本（避免修改原文件数据）
    new_reso_graph = ROOT.TGraphAsymmErrors()
    new_reso_graph.SetName("reso_syst")
    new_reso_graph.SetTitle("reso_syst (y error = 0.2% of y value)")
    
    for i in range(n_points):
        # 获取原始x/y值和x误差（x误差保持不变）
        x = reso_graph.GetX()[i]
        y = reso_graph.GetY()[i]
        x_err_low = reso_graph.GetEXlow()[i]
        x_err_high = reso_graph.GetEXhigh()[i]
        
        # 计算新的y误差：0.2% of y值（上下误差相同）
        y_err = abs(y) * 0.002  # 0.2% = 0.002
        y_err_low = y_err
        y_err_high = y_err
        
        # 设置新数据点和误差
        new_reso_graph.SetPoint(i, x, y)
        new_reso_graph.SetPointError(i, x_err_low, x_err_high, y_err_low, y_err_high)
        
        if i % 2 == 0:  # 每2个点打印一次进度
            print(f"  点{i}: x={x:.1f}, y={y:.4f}, 新y误差={y_err:.6f}")
    
    # 4. 重新计算tot_syst：总误差 = sqrt(fit² + reso² + fd²)
    print(f"\n开始重新计算tot_syst...")
    fit_graph = obj_dict["fit_syst"]
    fd_graph = obj_dict["fd_syst"]
    tot_graph = obj_dict["tot_syst"]
    
    # 检查各图数据点数量是否匹配（按x值匹配，而非索引）
    # 第一步：构建x值到数据的映射
    def build_x_map(graph):
        """构建x值到(索引, y, y_err_low, y_err_high)的映射"""
        x_map = {}
        n = graph.GetN()
        for i in range(n):
            x = graph.GetX()[i]
            y = graph.GetY()[i]
            y_err_low = graph.GetEYlow()[i]
            y_err_high = graph.GetEYhigh()[i]
            x_map[round(x, 1)] = (i, y, y_err_low, y_err_high)  # 四舍五入避免浮点误差
        return x_map
    
    # 构建各图的x值映射
    reso_x_map = build_x_map(new_reso_graph)
    fit_x_map = build_x_map(fit_graph)
    fd_x_map = build_x_map(fd_graph)
    
    # 收集所有有效x值（取交集）
    common_x = set(reso_x_map.keys()) & set(fit_x_map.keys()) & set(fd_x_map.keys())
    if not common_x:
        print("错误：reso_syst/fit_syst/fd_syst无共同的x值，无法计算tot_syst")
        in_file.Close()
        sys.exit(1)
    common_x = sorted(list(common_x))
    print(f"找到{len(common_x)}个共同的x值: {common_x}")
    
    # 创建新的tot_syst图
    new_tot_graph = ROOT.TGraphAsymmErrors()
    new_tot_graph.SetName("tot_syst")
    new_tot_graph.SetTitle("tot_syst (recalculated: sqrt(fit² + reso² + fd²))")
    
    for idx, x in enumerate(common_x):
        # 获取各系统误差的y误差（使用上误差，通常上下误差对称）
        _, _, fit_err, _ = fit_x_map[x]
        _, _, reso_err, _ = reso_x_map[x]
        _, _, fd_err, _ = fd_x_map[x]
        
        # 总误差 = 平方和开根号
        tot_err = (fit_err**2 + reso_err**2 + fd_err**2)**0.5
        
        # 获取x误差（从任意一个图取，假设x误差一致）
        reso_idx, _, _, _ = reso_x_map[x]
        x_err_low = new_reso_graph.GetEXlow()[reso_idx]
        x_err_high = new_reso_graph.GetEXhigh()[reso_idx]
        
        # tot_syst的y值保持与原tot_syst一致（或设为0，根据需求调整）
        if round(x,1) in build_x_map(tot_graph):
            tot_idx, tot_y, _, _ = build_x_map(tot_graph)[round(x,1)]
        else:
            tot_y = 0.0
        
        # 设置新tot_syst的数据点
        new_tot_graph.SetPoint(idx, x, tot_y)
        new_tot_graph.SetPointError(idx, x_err_low, x_err_high, tot_err, tot_err)
        
        print(f"  x={x:.1f}: fit_err={fit_err:.6f}, reso_err={reso_err:.6f}, fd_err={fd_err:.6f} → tot_err={tot_err:.6f}")
    
    # 5. 保存修改后的数据到输出文件
    try:
        out_file = ROOT.TFile.Open(output_file_path, "RECREATE")
        if not out_file or out_file.IsZombie():
            raise RuntimeError(f"无法创建输出文件: {output_file_path}")
        print(f"\n成功创建输出文件: {output_file_path}")
    except Exception as e:
        print(f"创建输出文件失败: {e}")
        in_file.Close()
        sys.exit(1)
    
    # 写入所有对象（保留原对象，替换修改后的reso_syst和tot_syst）
    out_file.cd()
    # 写入修改后的reso_syst
    new_reso_graph.Write()
    # 写入重新计算的tot_syst
    new_tot_graph.Write()
    # 写入其他未修改的对象
    obj_dict["fit_syst"].Write()
    obj_dict["fd_syst"].Write()
    # 如果有gvn_prompt_stat也一并写入
    gvn_stat = in_file.Get("gvn_prompt_stat")
    if gvn_stat:
        gvn_stat.Write()
    
    # 6. 关闭文件
    in_file.Close()
    out_file.Close()
    
    print(f"\n处理完成！输出文件: {output_file_path}")
    print("修改内容：")
    print("  1. reso_syst: 所有点y误差设为y值的0.2%")
    print("  2. tot_syst: 重新计算为 sqrt(fit_syst² + reso_syst² + fd_syst²)")

def main():
    # if len(sys.argv) != 3:
    #     print("用法: python adjust_syst.py <输入文件路径> <输出文件路径>")
    #     print("示例: python adjust_syst.py merged.root f1.root")
    #     sys.exit(1)
    
    input_path = '/home/xxf/cernbox/Lc/final-fig-paper/paper-fig/input-data/lc-d0-data/merged-lc-promptvn_withsyst.root'
    output_path = '/home/xxf/cernbox/Lc/final-fig-paper/paper-fig/input-data/lc-d0-data/merged-lc-promptvn_withsyst_r2-0.2%.root'
    adjust_reso_syst_and_recalc_tot(input_path, output_path)

if __name__ == "__main__":
    main()

成功打开输入文件: /home/xxf/cernbox/Lc/final-fig-paper/paper-fig/input-data/lc-d0-data/merged-lc-promptvn_withsyst.root
加载对象 reso_syst，数据点数量: 7
加载对象 fit_syst，数据点数量: 7
加载对象 fd_syst，数据点数量: 7
加载对象 tot_syst，数据点数量: 7

开始修改reso_syst的y误差（设置为y值的0.2%）...
  点0: x=2.5, y=0.1284, 新y误差=0.000257
  点2: x=4.5, y=0.1907, 新y误差=0.000381
  点4: x=7.0, y=0.1786, 新y误差=0.000357
  点6: x=18.0, y=0.1488, 新y误差=0.000298

开始重新计算tot_syst...
找到7个共同的x值: [2.5, 3.5, 4.5, 5.5, 7.0, 10.0, 18.0]
  x=2.5: fit_err=0.025000, reso_err=0.000257, fd_err=0.003654 → tot_err=0.025267
  x=3.5: fit_err=0.018000, reso_err=0.000335, fd_err=0.003405 → tot_err=0.018322
  x=4.5: fit_err=0.018000, reso_err=0.000381, fd_err=0.001500 → tot_err=0.018066
  x=5.5: fit_err=0.018000, reso_err=0.000416, fd_err=0.001500 → tot_err=0.018067
  x=7.0: fit_err=0.018000, reso_err=0.000357, fd_err=0.001500 → tot_err=0.018066
  x=10.0: fit_err=0.025000, reso_err=0.000285, fd_err=0.001500 → tot_err=0.025047
  x=18.0: fit_err=0.025000, reso_err=0.000298, fd_err=0.00

In [4]:
import ROOT
import sys
import math

def adjust_reso_syst_and_recalc_tot(input_file_path, output_file_path):
    """
    处理ROOT文件：
    1. 将reso_syst所有数据点的y误差设置为中心值（y值）的0.2%
    2. 重新计算tot_syst（总系统误差）：
       - 下误差：tot_err_low = sqrt(fit_err_low² + reso_err_low² + fd_err_low²)
       - 上误差：tot_err_high = sqrt(fit_err_high² + reso_err_high² + fd_err_high²)
    3. 保存修改后的数据到新文件
    
    参数:
        input_file_path: 输入ROOT文件路径（包含reso_syst/fit_syst/fd_syst/tot_syst等对象）
        output_file_path: 输出ROOT文件路径（保存修改后的数据）
    """
    # 初始化ROOT批处理模式（无GUI）
    ROOT.gROOT.SetBatch(True)
    
    # 1. 打开输入文件
    try:
        in_file = ROOT.TFile.Open(input_file_path, "READ")
        if not in_file or in_file.IsZombie():
            raise RuntimeError(f"无法打开输入文件: {input_file_path}")
        print(f"成功打开输入文件: {input_file_path}")
    except Exception as e:
        print(f"打开文件失败: {e}")
        sys.exit(1)
    
    # 2. 检查必要对象是否存在
    required_objs = ["reso_syst", "fit_syst", "fd_syst", "tot_syst"]
    obj_dict = {}
    for obj_name in required_objs:
        obj = in_file.Get(obj_name)
        if not obj or not isinstance(obj, ROOT.TGraphAsymmErrors):
            print(f"错误：输入文件中未找到有效对象 {obj_name} (TGraphAsymmErrors类型)")
            in_file.Close()
            sys.exit(1)
        obj_dict[obj_name] = obj
        print(f"加载对象 {obj_name}，数据点数量: {obj.GetN()}")
    
    # 3. 处理reso_syst：设置y误差为中心值的0.2%（非对称误差也设为对称的0.2%）
    reso_graph = obj_dict["reso_syst"]
    n_points = reso_graph.GetN()
    print(f"\n开始修改reso_syst的y误差（设置为y值的0.2%）...")
    
    # 创建修改后的reso_syst副本（避免修改原文件数据）
    new_reso_graph = ROOT.TGraphAsymmErrors()
    new_reso_graph.SetName("reso_syst")
    new_reso_graph.SetTitle("reso_syst (y error = 0.2% of y value)")
    
    reso_x_map = {}  # 存储x值到(索引, y, x_err_low, x_err_high, y_err_low, y_err_high)的映射
    for i in range(n_points):
        # 获取原始x/y值和x误差（x误差保持不变）
        x = reso_graph.GetX()[i]
        y = reso_graph.GetY()[i]
        x_err_low = reso_graph.GetEXlow()[i]
        x_err_high = reso_graph.GetEXhigh()[i]
        
        # 计算新的y误差：0.2% of y值（上下误差相同）
        y_err = abs(y) * 0.002  # 0.2% = 0.002
        y_err_low = y_err
        y_err_high = y_err
        
        # 设置新数据点和误差
        new_reso_graph.SetPoint(i, x, y)
        new_reso_graph.SetPointError(i, x_err_low, x_err_high, y_err_low, y_err_high)
        
        # 存入映射表（四舍五入避免浮点误差）
        reso_x_map[round(x, 3)] = (i, y, x_err_low, x_err_high, y_err_low, y_err_high)
        
        if i % 2 == 0:  # 每2个点打印一次进度
            print(f"  点{i}: x={x:.1f}, y={y:.4f}, 新y误差(下/上)={y_err_low:.6f}/{y_err_high:.6f}")
    
    # 4. 构建其他系统误差的x值映射（保留非对称误差）
    def build_asymm_x_map(graph):
        """构建x值到(索引, y, x_err_low, x_err_high, y_err_low, y_err_high)的映射"""
        x_map = {}
        n = graph.GetN()
        for i in range(n):
            x = graph.GetX()[i]
            y = graph.GetY()[i]
            x_err_low = graph.GetEXlow()[i]
            x_err_high = graph.GetEXhigh()[i]
            y_err_low = graph.GetEYlow()[i]
            y_err_high = graph.GetEYhigh()[i]
            x_map[round(x, 3)] = (i, y, x_err_low, x_err_high, y_err_low, y_err_high)
        return x_map
    
    fit_x_map = build_asymm_x_map(obj_dict["fit_syst"])
    fd_x_map = build_asymm_x_map(obj_dict["fd_syst"])
    tot_x_map = build_asymm_x_map(obj_dict["tot_syst"])
    
    # 收集所有有效x值（取交集）
    common_x = set(reso_x_map.keys()) & set(fit_x_map.keys()) & set(fd_x_map.keys())
    if not common_x:
        print("错误：reso_syst/fit_syst/fd_syst无共同的x值，无法计算tot_syst")
        in_file.Close()
        sys.exit(1)
    common_x = sorted(list(common_x))
    print(f"\n找到{len(common_x)}个共同的x值: {[round(x,1) for x in common_x]}")
    
    # 5. 重新计算tot_syst（非对称误差）
    print(f"\n开始重新计算tot_syst（非对称误差）...")
    new_tot_graph = ROOT.TGraphAsymmErrors()
    new_tot_graph.SetName("tot_syst")
    new_tot_graph.SetTitle("tot_syst (recalculated: sqrt(fit² + reso² + fd²) 非对称误差)")
    
    for idx, x_key in enumerate(common_x):
        x = float(x_key)
        
        # 获取各系统误差的非对称y误差
        # reso_syst（已修改）
        _, reso_y, reso_xl, reso_xh, reso_yl, reso_yh = reso_x_map[x_key]
        # fit_syst（原始非对称误差）
        _, fit_y, fit_xl, fit_xh, fit_yl, fit_yh = fit_x_map[x_key]
        # fd_syst（原始非对称误差）
        _, fd_y, fd_xl, fd_xh, fd_yl, fd_yh = fd_x_map[x_key]
        
        # 核心：非对称总误差计算（平方和开根号）
        # 下误差：tot_err_low = sqrt(fit_err_low² + reso_err_low² + fd_err_low²)
        tot_err_low = math.sqrt(fit_yl**2 + reso_yl**2 + fd_yl**2)
        # 上误差：tot_err_high = sqrt(fit_err_high² + reso_err_high² + fd_err_high²)
        tot_err_high = math.sqrt(fit_yh**2 + reso_yh**2 + fd_yh**2)
        
        # tot_syst的y值保持与原tot_syst一致（若存在）
        if x_key in tot_x_map:
            tot_y = tot_x_map[x_key][1]
        else:
            tot_y = 0.0
        
        # x误差：取reso_syst的x误差（或取平均值，根据需求调整）
        tot_x_err_low = reso_xl
        tot_x_err_high = reso_xh
        
        # 设置新tot_syst的数据点（非对称误差）
        new_tot_graph.SetPoint(idx, x, tot_y)
        new_tot_graph.SetPointError(
            idx,
            tot_x_err_low,   # x下误差
            tot_x_err_high,  # x上误差
            tot_err_low,     # y下误差（非对称）
            tot_err_high     # y上误差（非对称）
        )
        
        print(f"  x={x:.1f}: ")
        print(f"    fit误差(下/上)={fit_yl:.6f}/{fit_yh:.6f}")
        print(f"    reso误差(下/上)={reso_yl:.6f}/{reso_yh:.6f}")
        print(f"    fd误差(下/上)={fd_yl:.6f}/{fd_yh:.6f}")
        print(f"    → tot误差(下/上)={tot_err_low:.6f}/{tot_err_high:.6f}")
    
    # 6. 保存修改后的数据到输出文件
    try:
        out_file = ROOT.TFile.Open(output_file_path, "RECREATE")
        if not out_file or out_file.IsZombie():
            raise RuntimeError(f"无法创建输出文件: {output_file_path}")
        print(f"\n成功创建输出文件: {output_file_path}")
    except Exception as e:
        print(f"创建输出文件失败: {e}")
        in_file.Close()
        sys.exit(1)
    
    # 写入所有对象（保留原对象，替换修改后的reso_syst和tot_syst）
    out_file.cd()
    # 写入修改后的reso_syst
    new_reso_graph.Write()
    # 写入重新计算的tot_syst（非对称误差）
    new_tot_graph.Write()
    # 写入其他未修改的对象
    obj_dict["fit_syst"].Write()
    obj_dict["fd_syst"].Write()
    # 如果有gvn_prompt_stat也一并写入
    gvn_stat = in_file.Get("gvn_prompt_stat")
    if gvn_stat and isinstance(gvn_stat, ROOT.TGraphAsymmErrors):
        gvn_stat.Write()
    
    # 7. 关闭文件
    in_file.Close()
    out_file.Close()
    
    print(f"\n处理完成！输出文件: {output_file_path}")
    print("修改内容：")
    print("  1. reso_syst: 所有点y误差设为y值的0.2%（对称）")
    print("  2. tot_syst: 重新计算非对称误差：")
    print("     - 下误差 = √(fit_err_low² + reso_err_low² + fd_err_low²)")
    print("     - 上误差 = √(fit_err_high² + reso_err_high² + fd_err_high²)")

def main():
    # if len(sys.argv) != 3:
        # print("用法: python adjust_syst.py <输入文件路径> <输出文件路径>")
        # print("示例: python adjust_syst.py merged.root f1.root")
        # sys.exit(1)
    
    input_path = '/home/xxf/cernbox/Lc/final-fig-paper/paper-fig/input-data/lc-d0-data/merged-lc-promptvn_withsyst.root'
    output_path = '/home/xxf/cernbox/Lc/final-fig-paper/paper-fig/input-data/lc-d0-data/merged-lc-promptvn_withsyst_r2-0.2%.root'
    adjust_reso_syst_and_recalc_tot(input_path, output_path)

if __name__ == "__main__":
    main()

成功打开输入文件: /home/xxf/cernbox/Lc/final-fig-paper/paper-fig/input-data/lc-d0-data/merged-lc-promptvn_withsyst.root
加载对象 reso_syst，数据点数量: 7
加载对象 fit_syst，数据点数量: 7
加载对象 fd_syst，数据点数量: 7
加载对象 tot_syst，数据点数量: 7

开始修改reso_syst的y误差（设置为y值的0.2%）...
  点0: x=2.5, y=0.1284, 新y误差(下/上)=0.000257/0.000257
  点2: x=4.5, y=0.1907, 新y误差(下/上)=0.000381/0.000381
  点4: x=7.0, y=0.1786, 新y误差(下/上)=0.000357/0.000357
  点6: x=18.0, y=0.1488, 新y误差(下/上)=0.000298/0.000298

找到7个共同的x值: [2.5, 3.5, 4.5, 5.5, 7.0, 10.0, 18.0]

开始重新计算tot_syst（非对称误差）...
  x=2.5: 
    fit误差(下/上)=0.025000/0.025000
    reso误差(下/上)=0.000257/0.000257
    fd误差(下/上)=0.003654/0.005396
    → tot误差(下/上)=0.025267/0.025577
  x=3.5: 
    fit误差(下/上)=0.018000/0.018000
    reso误差(下/上)=0.000335/0.000335
    fd误差(下/上)=0.003405/0.004530
    → tot误差(下/上)=0.018322/0.018564
  x=4.5: 
    fit误差(下/上)=0.018000/0.018000
    reso误差(下/上)=0.000381/0.000381
    fd误差(下/上)=0.001500/0.001500
    → tot误差(下/上)=0.018066/0.018066
  x=5.5: 
    fit误差(下/上)=0.018000/0.018000
    re